# 인공신경망(ANN)

In [13]:
import torch #핵심 라이브러리
import torch.nn as nn #신경망 구축을 위한 기능
import torch.optim as optim #최적화
import torchvision #입력이 이미지 형식일 떄 이미지 처리-전처리
import torchvision.transforms as transforms

In [14]:
#데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), 0.5,)
])
# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [15]:
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=0.5)
           )

In [16]:
trainloader

In [20]:
class SimpleANN(nn.Module): #nn.Module에서 기능 상속
    def __init__(self): #__init__은 클래스가 생성되는 순간, 자동으로 호출
        super(SimpleANN, self).__init__()#부모 클래스에서 기능을 작동하는 super 함수
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로, 28*28: ㅡ데이터의 크기
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로, relu: 활성화 함수
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x): #forward 함수라고 이름을 지어주면, 레이어간의 연결 관계를 설정해줄 수 있음.
         x = x.view(-1, 28*28) #입력 이미지를 1차원 벡터로 변환, -1: 28*28 차원을 기준으로 데이터를 알아서 결정해라
         x = torch.relu(self.fc1(x))
         x = torch.relu(self.fc2(x))
         x = self.fc3(x)
         return x

In [23]:
#모델 학습

# 모델 초기화
model = SimpleANN()


correct = 0
total = 0
with torch.no_grad(): #no_grad(): 메모리 효율성을 위해 사용, 평가 단계에서 기울기를 계산하지 않는다.
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 10.59%


In [27]:
# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss() #CrossEntropyLoss:분류 모델에서 성능을 평가하기 위해 사용되는 손실 함수 중 하나
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #SGD: 손실함수를 미적분해서 가중치를 높인다. -> 

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() #기울기를 기준으로 가중치를 업데이트

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 0.037
[Epoch 1, Batch 200] loss: 0.036
[Epoch 1, Batch 300] loss: 0.037
[Epoch 1, Batch 400] loss: 0.032
[Epoch 1, Batch 500] loss: 0.035
[Epoch 1, Batch 600] loss: 0.044
[Epoch 1, Batch 700] loss: 0.040
[Epoch 1, Batch 800] loss: 0.045
[Epoch 1, Batch 900] loss: 0.038
[Epoch 2, Batch 100] loss: 0.040
[Epoch 2, Batch 200] loss: 0.037
[Epoch 2, Batch 300] loss: 0.042
[Epoch 2, Batch 400] loss: 0.034
[Epoch 2, Batch 500] loss: 0.036
[Epoch 2, Batch 600] loss: 0.044
[Epoch 2, Batch 700] loss: 0.037
[Epoch 2, Batch 800] loss: 0.035
[Epoch 2, Batch 900] loss: 0.050
[Epoch 3, Batch 100] loss: 0.029
[Epoch 3, Batch 200] loss: 0.035
[Epoch 3, Batch 300] loss: 0.031
[Epoch 3, Batch 400] loss: 0.028
[Epoch 3, Batch 500] loss: 0.032
[Epoch 3, Batch 600] loss: 0.031
[Epoch 3, Batch 700] loss: 0.030
[Epoch 3, Batch 800] loss: 0.033
[Epoch 3, Batch 900] loss: 0.043
[Epoch 4, Batch 100] loss: 0.031
[Epoch 4, Batch 200] loss: 0.023
[Epoch 4, Batch 300] loss: 0.030
[Epoch 4, 

In [30]:
#모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(images, labels)

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., 